In [1]:
import networkx as nx

model = nx.DiGraph([
            ("C", "A"),
            ("C", "B"),
            ("D", "A"),
            ("B", "E"),
            ("F", "E"),
            ("A", "G"),
        ])

[ee](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098140243/files/assets/ciip_03in11.png )

In [8]:
print("Are G and F dependent?")
print(not(nx.d_separated(model, {"G"}, {"F"}, {})))

Are G and F dependent?
False
